In [ ]:
import pandas as pd
import numpy as np
import csv, json

# create csv 
read_file = pd.read_excel(r'/Users/os/Documents/Cindy_Synthea/eMERGE_List.xlsx', sheet_name='NUJHU Positive Sample List')
read_file.to_csv (r'/Users/os/Documents/Synthea/eMERGE_List.csv', index = None, header=True)
# thewriter=csv.writer('/Users/os/Documents/Synthea/eMERGE_List.csv')

# modify csv for easier use later on
df = pd.read_csv('/Users/os/Documents/Synthea/eMERGE_List.csv')
for index in range(len(df)):
    if df[index, 5] == 'Male':
        df.at[index, 5] = M
    if df[index, 5] == 'Female':
        df.at[index, 5] = F
    if df[index, 6] == 'Colorectal Cancer / Polyps':
        df.at[index, 6] == 'colon cancer'
    if df[index, 6] == 'Breast carcinoma':
        df.at[index, 6] == 'breast cancer'
    if df[index, 6] == 'Ovarian Cancer, Epithelial, Included':
        df.at[index, 6] == 'ovarian cancer'
    if df[index, 6] == 'Not selected for trait' or indication == 'Healthy':
        df.at[index, 6] == 'NONE'
    # add more if searching for some is hard
df["Freq of Unique Codes"] = ""
df.to_csv('/Users/os/Documents/Synthea/eMERGE_List.csv', index=False)


# Find all value of specified key of nested JSON into array 
def extract_values(obj, key):
    arr = []
    def extract(obj, arr, key):
        # Recursively search for values of key in JSON tree
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

# access ohdsi API to find all SNOMED codes
def findSNOMED_ohdsi(txt): 
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": txt,
              "VOCABULARY_ID": ['SNOMED']}
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(data.decode("utf-8"))
    return [d["CONCEPT_CODE"] for d in data]


# create array of info
info = np.genfromtxt('/Users/os/Documents/Synthea/eMERGE_List.csv', delimiter=',',skip_header=0)
    

# create dictionary of indications and corresponding SNOMED codes
unique_indications = set(info[6])
codes_indications = dict.fromkeys(unique_indications, [])
for key, val in codes_indications:
    if key == 'NONE': val == [162673000]
    else: val = findSNOMED_ohdsi(key)
        
        
# create place to store all the eventual directories
all_output = r'/Users/os/Documents/Synthea/all' 
if not os.path.exists(all_output):
    os.makedirs(all_output)
accepted_output = r'/Users/os/Documents/Synthea/accepted' 
if not os.path.exists(accepted_output):
    os.makedirs(accepted_output)
    
# loop through each patient    
for index, row_content in enumerate(info):
    row_output = r'/Users/os/Documents/Synthea/all/' + index
    if not os.path.exists(row_output):
        os.makedirs(row_output)
    # find match for each patient
    check = True:
        while check:
            os.chdir(row_output)
            os.system(' '.join['java', '-jar', '/Users/os/Documents/Synthea/synthea-with-dependencies.jar','-p 10', 
                              '-a ' + row_content[3] + '-' + row_content[3], '-g ' + row_content[5]])
            filelist = os.listdir(row_output+'/output/fhir')
            for file in filelist:
                with codecs.open(row_output+'/output/fhir/'+ file, 'r', encoding='utf8', errors='replace') as f:
                    json_text = json.load(f)
                    race = json_text['entry'][0]['resource']['extension'][0]['extension'][1]['valueString']
                    if race == row_content[4]:
                        # CHANGE KEYCHANGETHIS TO BE CODE KEY
                        codes = extract_values(f, KEYCHANGETHIS)
                        for code in codes:
                            if len(code) != 9 or !code.isdigit():
                                codes.pop(code)
                        if set(codes) & set(codes_indications[row_content[6]])
                            # WRITE CODE TO ACCEPT THIS and move to folder
                            # WRITE ID TO DF
                            # MOVE COPY OF FILE TO accepted_output
                            # PROBABLY DELETE OTHER FILES
                    os.remove(row_output + '/' + file)
                            
jsons_df.to_csv(r'/Users/cindy/Documents/Synthea/output/fhir/match_df.csv')